In [ ]:
import cv2 as cv
import numpy as np
import time
import matplotlib.pyplot as plt
import warnings
warnings.warn("deprecated", DeprecationWarning)

In [ ]:
def video_to_array(path):
    cap = cv.VideoCapture(path)
    frameCount = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
    buf = np.empty((frameCount, 400, 640, 3), np.dtype('uint8'))

    fc = 0
    ret = True

    while (fc < frameCount  and ret):
        ret, frame = cap.read()
        buf[fc] = cv.resize(frame, (640, 400))
        fc += 1

    cap.release()
    return buf


        
    

In [ ]:
images = video_to_array('Untitled.mp4')


In [55]:
backSub = cv.createBackgroundSubtractorMOG2()
for img in images:
    hsv = cv.cvtColor(img,cv.COLOR_RGB2LAB)
    fgMask = backSub.apply(hsv[:,:,1])
    _,threshold = cv.threshold(fgMask, 180, 255, cv.THRESH_BINARY)
    contours, _ = cv.findContours(threshold, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    i = 0
  
    # list for storing names of shapes
    for contour in contours:
    
        # here we are ignoring first counter because 
        # findcontour function detects whole image as shape
        if i == 0:
            i = 1
            continue
    
        # cv2.approxPloyDP() function to approximate the shape
        approx = cv.approxPolyDP(contour, 0.01 * cv.arcLength(contour, True), True)
        cv.drawContours(img, [contour], 0, (0, 0, 255), 5)
            # finding center point of shape
        M = cv.moments(contour)
        if M['m00'] != 0.0:
            x = int(M['m10']/M['m00'])
            y = int(M['m01']/M['m00'])
    
            # putting shape name at center of each shape
            if len(approx) == 3:
                pass
        
            elif len(approx) == 4:
                pass
        
            elif len(approx) == 5:
                pass
        
            elif len(approx) == 6:
                pass
        
            else:
                cv.putText(img, 'circle', (x, y),
                        cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    cv.imshow('img',img)
    cv.imshow('back',fgMask)
    cv.waitKey(0)
cv.destroyAllWindows()

KeyboardInterrupt: 

In [52]:
threshold.shape

  


(400, 640)